In [ ]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
from dollarpy import Recognizer, Template, Point
# import csv
import os

import numpy as np
import matplotlib.pyplot as plt
import bluetooth 
import socket
import pickle 
import dlib


import imutils
from imutils import face_utils
from deepface import DeepFace
from GazeTracking import gaze_tracking 


mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [ ]:
def saveBlue(sendd):
    directory = r"C:\Users\LENOVO-2021\OneDrive\Desktop\TUIO11_NET-master\bin\Debug"

    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Set the file name and file path
    file_name = "Text.txt"
    file_path = os.path.join(directory, file_name)

    # Specify the string to save to the file
    string_to_save = sendd

    # Open the file in write mode and save the string
    with open(file_path, "w") as f:
        f.write(string_to_save+'\n')

    # Print a confirmation message
    print(f"The string has been saved to {file_path}.")


In [ ]:
def rightXY(sendd):
    directory = r"C:\Users\LENOVO-2021\OneDrive\Desktop\TUIO11_NET-master\bin\Debug"

    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Set the file name and file path
    file_name = "RIGHTxy.txt"
    file_path = os.path.join(directory, file_name)

    # Specify the string to save to the file
    string_to_save = sendd

    # Open the file in write mode and save the string
    with open(file_path, "w") as f:
        #f.write(string_to_save)
        f.writelines(string_to_save)
    # Print a confirmation message
    print(f"The string has been saved to {file_path}.")


In [ ]:
def leftXY(sendd):
    directory = r"C:\Users\LENOVO-2021\OneDrive\Desktop\TUIO11_NET-master\bin\Debug"

    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Set the file name and file path
    file_name = "LEFTxy.txt"
    file_path = os.path.join(directory, file_name)

    # Specify the string to save to the file
    string_to_save = sendd

    # Open the file in write mode and save the string
    with open(file_path, "w") as f:
        f.write(string_to_save)

    # Print a confirmation message
    print(f"The string has been saved to {file_path}.")


In [ ]:
def bluetoothh():
    listBlutooth=[]
    print("Searching.....")
    nearby_devices = bluetooth.discover_devices(lookup_names=True)
    print("found %d devices" % len(nearby_devices))
    
    for addr, name in nearby_devices:
        print(" %s - %s" % (addr, name))
        
        listBlutooth.append(name)
    return listBlutooth


In [ ]:
#if res=="true":
soc = socket.socket()
hostname="localhost"# 127.0.0.1 #0.0.0.0
port=5000
soc.bind((hostname,port))
soc.listen(5)
conn , addr = soc.accept()
print("device connected")

In [ ]:
#if res=="true":
send=bluetoothh()
sendd=str(send)
saveBlue(sendd)
msg=bytes(sendd,'utf-8')
conn.send(msg)

In [ ]:
templates=[] #list of templates for $1 training


In [ ]:
sendRight=''
sendLeft=''

In [ ]:
def getPoints(videoURL,label):
    cap = cv2.VideoCapture(videoURL)#web cam =0 , else enter filename
    # Initiate holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        #List to hold Coordinates
        points = []
        palm=[]
        
        #List to Plot
        m_palm=[]
    
        while cap.isOpened():
            ret, frame = cap.read()

            # Recolor Feed
            if ret==True:

                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False        
                
                # Make Detections
                results = holistic.process(image)
                # print(results.face_landmarks)

                # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

                # Recolor image back to BGR for rendering
                image.flags.writeable = True   
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


                # 2. Right hand

                # 5. Palm center
                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                                     mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                                     mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                                     )
                if results.right_hand_landmarks:
                    # Extract palm center
                    x_center = int(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].x * image.shape[1])
                    y_center = int(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].y * image.shape[0])

                    # add point of palm center
                    palm.append(Point(results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].x,
                                      results.right_hand_landmarks.landmark[mp_holistic.HandLandmark.WRIST].y,1))
                    m_palm.append((x_center, y_center))
                # Export coordinates
                try:
                    # Extract Pose landmarks
                    pose = results.pose_landmarks.landmark
                    index = 0
                    newlist=[]
                    for lnd in pose:
                        if(index  in [12]):
                            newlist.append(lnd)
                        index+=1



                    # add points of wrist , elbow and shoulder
                    palm.append(Point(newlist[0].x,newlist[0].y,1))

                   
                    m_palm.append((newlist[9].x,newlist[9].y))
                   
                    #Pose Landmarks
                    #pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in newlist]).flatten())

                    # Extract Face landmarks
                    #face = results.face_landmarks.landmark

                    # Concate rows
                    #row = pose_row
                    points = palm

                    recognizer = Recognizer(templates)
                    result = recognizer.recognize(points)
                    send=str(result[0])+","
                    msg=bytes(send,'utf-8')
                    conn.send(msg)


                ###########################################
   
                except:
                    pass

                cv2.imshow(label, image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()
    print(label)
    
    return points

In [ ]:
#hook correct
#if res=="true":
def dssa(s):
    #if res=="true":
    soc = socket.socket()
    hostname="localhost"# 127.0.0.1 #0.0.0.0
    port=5001
    soc.bind((hostname,port))
    soc.listen(5)
    conn , addr = soc.accept()
    print("device connected")

    vid = "Right2.mp4"
    #points = getPoints(vid,"Right") 
    tmpl_2 = Template('Right', points)
    templates.append(tmpl_2)
    print (points)
    rightXY(str(points))
        #UPPER
        #uppercut correct
    vid = "Left 2.mp4"
    points = getPoints(vid,"Left") 
    tmpl_2 = Template('Left', points)
    templates.append(tmpl_2)
    leftXY(str(points))


In [ ]:

def faceX(videoURL):
    face_detector = dlib.get_frontal_face_detector()
    landmark_detector = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')
    cam = cv2.VideoCapture(videoURL)  # videoURL can be a video file path or a URL

    while True:
        ret, image = cam.read()
        if ret:
            image = imutils.resize(image, width=500)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            rects = face_detector(gray, 1)

            for (i, rect) in enumerate(rects):
                landmarks = landmark_detector(gray, rect)
                landmarks = face_utils.shape_to_np(landmarks)
                (x, y, w, h) = face_utils.rect_to_bb(rect)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(image, f"Face {i + 1}", (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                for (x, y) in landmarks:
                    cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
                    
            cv2.imshow('Live feed', image)
    
        if cv2.waitKey(1) == 27:  # Press Esc to exit
            break

    cam.release()
    cv2.destroyAllWindows()

# Call the function with the video URL

In [ ]:
vid = "http://192.168.8.100:8080//video"

#



In [ ]:
def deepfaceX(url):
    model = DeepFace.build_model("Emotion")

    
    cap = cv2.VideoCapture(url)  

    if not cap.isOpened():
        print("Error opening video")
        return

    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 1

    while True:
        ret, frame = cap.read()

        if ret:
            faces = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)

            for face in faces:
                emotion_label = face['emotion']
                flag=0
                fear_probability = emotion_label['fear']
                neutral_probability = emotion_label['neutral']
                if fear_probability>70:
                    flag=1
                fear_probability_str = str(fear_probability)
                neutral_probability_str = str(neutral_probability)
                sendd=str(flag)
                print (sendd)
                msg=bytes(sendd,'utf-8')
                conn.send(msg)
                cv2.putText(frame, "Fear: " + fear_probability_str, (10, 30), font, scale, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, "Neutral: " + neutral_probability_str, (10, 60), font, scale, (0, 255, 0), 2, cv2.LINE_AA)

            cv2.imshow('Video', frame)

        # Check for the 'q' key to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoCapture object and close the window
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
#if res=="true":
soc = socket.socket()
hostname="localhost"# 127.0.0.1 #0.0.0.0
port=5003
soc.bind((hostname,port))
soc.listen(5)
conn , addr = soc.accept()
print("device connected")


In [ ]:
#if res=="true":
vid = "http://192.168.8.100:8080//video"
deepfaceX(0)
faceX(0)

In [ ]:
#TEST
#if res=="true":
vid = "http://192.168.8.100:8080//video"
points = getPoints(0,"Test") 
import time 
start = time.time()

#print (points)
end = time.time()
duration=end-start
    #print(result[0])
    #print(duration)
    #print (recognizer)


In [ ]:
import time 

def gazeT(url):
       # Initialize the GazeTracking object
    gaze = gz.GazeTracking()

    # Open the webcam
    cap = cv2.VideoCapture(0)

    while True:
        # Read a frame from the webcam
        _, frame = cap.read()

        # Flip the frame horizontally for intuitive display
        frame = cv2.flip(frame, 1)

        # Detect the gaze
        gaze.refresh(frame)
        
        duration=end-start
        # Get the estimated gaze position
        left_pupil = gaze.pupil_left_coords()
        right_pupil = gaze.pupil_right_coords()
        flag=0
        # Determine eye gaze direction
        if gaze.is_center():
            gaze_direction = "Center"
            flag=0
        elif gaze.is_left():
            gaze_direction = "Left"
            start = time.time()
            
            flag=1
        elif gaze.is_right():
            gaze_direction = "Right"
            start = time.time()
            
            flag=1
        else:
            gaze_direction = "Undetermined"
        end = time.time()
        duration=end-start
        if duration>=5 and flag==1:
            flag=1
            duration=0
            sendd=str(flag)
            msg=bytes(sendd,'utf-8')
            conn.send(msg)        # Draw the gaze landmarks and direction on the frame
        frame = gaze.annotated_frame()
        cv2.putText(frame, f"Gaze Direction: {gaze_direction}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Display the frame
        cv2.imshow("Gaze Tracking", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
#if res=="true":
soc = socket.socket()
hostname="localhost"# 127.0.0.1 #0.0.0.0
port=5013
soc.bind((hostname,port))
soc.listen(5)
conn , addr = soc.accept()
print("device connected")
from GazeTracking.gaze_tracking import gaze_tracking as gz
gazeT(0)


In [ ]:
import cv2
import numpy as np
import imutils
def track():
    cap = cv2.VideoCapture(0)
    prev_centroid = None
    countup=0
    countdw=0
    countL=0
    countR=0
    count=0
    while True:
        ret, frame = cap.read()

        if frame is None:
            break

        blurred = cv2.GaussianBlur(frame, (5, 5), 0)
        hsv=cv2.cvtColor(blurred,cv2.COLOR_BGR2HSV)

        Low_Blue=np.array([40,100,100])
        Upr_Blue=np.array([120,255,255])
        mask=cv2.inRange(hsv,Low_Blue,Upr_Blue)

        contours, _ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
        current_centroid = None
        for contour in contours:
            area=cv2.contourArea(contour)
            if area > 1000:
                c = max(contours, key=cv2.contourArea)
                ((x, y), radius) = cv2.minEnclosingCircle(c)
                M = cv2.moments(c)
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                cv2.drawContours(frame,contour,-1,(0,255,0),3)
                current_centroid=(center)
                cv2.circle(frame, center, 5, (0, 0, 255), -1)

                if prev_centroid is not None:
                    # Calculate the movement direction based on the centroid positions
                    diff_x = current_centroid[0] - prev_centroid[0]
                    diff_y = current_centroid[1] - prev_centroid[1]
                    send=str(current_centroid[0])+","+str(current_centroid[1])
                    msg=bytes(send,'utf-8')
                    conn.send(msg)


                    if diff_y > 30:
                        movement = "Object moved down"
                        countdw+=1
                        count=countdw
                    elif diff_y < -30:
                        movement = "Object moved up"
                        countup+=1
                        count=countup
                    elif diff_x > 30:
                        movement = "Object moved right"
                        countR+=1
                        count=countR
                    elif diff_x < -30:                    
                        movement = "Object moved left"
                        countL+=1
                        count=countL
                    else:
                        movement = "Object is stationary"

                    # Display the movement direction
                    cv2.putText(frame, movement+str(count), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                prev_centroid = current_centroid 
        #cv2.drawContours(frame,contours,-1,(0,255,0),3)
        resized_frame = cv2.resize(frame, (1000, 800))
        cv2.imshow('Object Tracking', resized_frame)
        cv2.imshow('Mask',mask)

        key= cv2.waitKey(1) 
        if key == 27 & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
#if res=="true":
soc = socket.socket()
hostname="localhost"# 127.0.0.1 #0.0.0.0
port=5007
soc.bind((hostname,port))
soc.listen(5)
conn , addr = soc.accept()
print("device connected")